# Lab 2: Set Covering with EA

In [12]:
import random
import logging
import collections
import numpy as np
import math

In [13]:
def problem(N, seed=None):
    random.seed(seed)
    array = [
        list(set(random.randint(0, N - 1) for n in range(random.randint(N // 5, N // 2))))
        for n in range(random.randint(N, N * 5))
    ]
    ranked_list = [(sorted(x),len(x)) for x in array]
    return ranked_list

In [14]:
def better(old,new,discovered_Nums):
    if len(new)< len(old):
        # w is less for sure so we select the new node
        return True
    elif len(new) >= len(old):
        newCover = len(collections.Counter(new) - collections.Counter(discovered_Nums))
        oldCover = len(collections.Counter(old) - collections.Counter(discovered_Nums))
        if newCover > oldCover:
            return True
        else: 
            return False
    else:
        return False
    
def bestNode(betterNodes,discovered_Nums):
    best= betterNodes[0]
    for i in range(len(betterNodes)):
        if len(best)>= len(betterNodes[i]):
            bestCover = len(collections.Counter(best) - collections.Counter(discovered_Nums))
            nodeCover = len(collections.Counter(betterNodes[i]) - collections.Counter(discovered_Nums))
            if nodeCover >= bestCover:
                best = betterNodes[i]
        else:
            bestCover = len(collections.Counter(best) - collections.Counter(discovered_Nums))
            nodeCover = len(collections.Counter(betterNodes[i]) - collections.Counter(discovered_Nums))
            if nodeCover == bestCover:
                best = betterNodes[i]
    return best


In [34]:
def Tw_1p1(state,stateOldNums,N):
    sigma = 0.01 *N
    newState =[]
    newNums =[]
    if not stateOldNums:
        return sorted(state)
    state = list((collections.Counter(state) - collections.Counter(stateOldNums)).keys())
    while len(set(newState)) != (len(stateOldNums)+len(state)):
        # the while loop guarantees that for us we keep producing multiple generation until
        # we end up with a generation with the best predictable performace
        newNums = np.random.normal(loc=stateOldNums,scale=sigma,size=(len(stateOldNums),))
        newNums = [math.ceil(x) for x in newNums]
        newNums = np.clip(newNums, a_min=0, a_max=None)
        newState = state.copy()
        newState.extend(newNums)
    return sorted(newState)
    
def Tw_1plmda(state,stateOldNums,discovered_Nums,N):
    lmda = math.ceil(0.01 *N)
    bestOfNodes = state
    offspring = list()
    offspring.append(state)
    for _ in range(lmda):
        offspring.append(Tw_1p1(state,stateOldNums,N))
    
    if len(offspring) !=0:
        bestOfNodes= bestNode(offspring,discovered_Nums)
    else:
        bestOfNodes = state 
    return bestOfNodes

def Tw_1clmda(state,stateOldNums,discovered_Nums,N):
    lmda = math.ceil(0.2 *N)
    bestOfNodes = state
    offspring = list()
    for _ in range(lmda):
        offspring.append(Tw_1p1(state,stateOldNums,N))

    if len(offspring) !=0:
        bestOfNodes= bestNode(offspring,discovered_Nums)
    else:
        bestOfNodes = state 
    return bestOfNodes

def Tw_uclmda(nodes,discovered_Nums,N):
    u = math.ceil(0.4* N)
    lmda = math.ceil(0.7 *N)
    indexes = np.random.randint(0,u,size =(lmda,))

    ind = list(indexes)
    newOffSpring=[nodes[i] for i in ind]
    nodes= np.array(nodes,dtype = object)
    offSpring = nodes[indexes][:,0]
    
    nodes = np.delete(nodes,indexes)

    
    for state in offSpring:
        stateNewNums = collections.Counter(state) - collections.Counter(discovered_Nums)
        stateOldNums = tuple((collections.Counter(state)- stateNewNums).keys())
        newstate =Tw_1p1(state,stateOldNums,N)
        newOffSpring.append((newstate,len((collections.Counter(newstate)-collections.Counter(discovered_Nums)).keys())))
    newOffSpring = sorted(newOffSpring,key=lambda x:x[1])
    
    return newOffSpring[0:u]


In [17]:
def Tweak(state,nodes,discovered_Nums,N,mtype):
    stateNewNums = collections.Counter(state) - collections.Counter(discovered_Nums)
    stateOldNums = tuple((collections.Counter(state)- stateNewNums).keys())
    newState = state
    if (mtype == '1p1'):
        newState = Tw_1p1(state,stateOldNums,N)
        if not better(state,newState,discovered_Nums):
            newState= state
    elif (mtype == '1plmda'):
        newState = Tw_1plmda(state,stateOldNums,discovered_Nums,N)  
    elif (mtype == '1clmda'):
        newState = Tw_1clmda(state,stateOldNums,discovered_Nums,N)
    elif (mtype == 'uclmda'):
        newState = Tw_uclmda(nodes,discovered_Nums,N)
    else:
        print('unvalid Algorithm')

    
    return newState
    

In [18]:
# "1p1" -> 1+1
# "1plmda" -> 1 + Lambda
# "1clmda" -> 1 , Lambda
# "uclmda" -> u , Lambda
def evolutionary_Strategy(nodes,N,seed =42,mtype ="1p1"):
      random.seed(seed)
      discovered_Nums=list()
      st_Index = random.randint(0, len(nodes)-1)
      st_node = nodes[st_Index][0]
      solution_nodes = list()
      
      if mtype == 'uclmda':
            for _ in range(len(nodes)):
                  nwnodes= Tweak(state=st_node,nodes=nodes,
                  discovered_Nums=discovered_Nums,
                  N=N,
                  mtype=mtype)
                  best_node = nwnodes[0][0]
                  for node in nwnodes:
                        if better(best_node,node[0],discovered_Nums):
                              best_node= node[0]
                  discovered_Nums.extend(list((collections.Counter(best_node)-collections.Counter(discovered_Nums)).keys()))
                  solution_nodes.append(best_node)
                  if len(discovered_Nums) == N:
                        break
                  if not nodes:
                        print("Algorithm failed to find any solution")
                        return None
      else:

            for _ in range(len(nodes)):
                  nwnode= Tweak(state=st_node,nodes=nodes,
                        discovered_Nums=discovered_Nums,
                        N=N,
                        mtype=mtype)
                  discovered_Nums.extend(list((collections.Counter(nwnode)-collections.Counter(discovered_Nums)).keys()))
                  solution_nodes.append(nwnode)
                  
                  if len(discovered_Nums) == N:
                        break
                  
                  nodes.remove(tuple([sorted(st_node),len(st_node)]))
                  if not nodes:
                        print("Algorithm failed to find any solution")
                        return None
                  st_Index = random.randint(0, len(nodes)-1)
                  st_node = nodes[st_Index][0]
      weight = sum(len(x) for x in solution_nodes)
      print(f"For N = {N} => the list has a weight W = {weight} and we used {len(solution_nodes)} nodes to cover it")
      

In [19]:
STRATEGY_TYPE = {'1 + 1': "1p1" , '1 + Lambda':"1plmda" ,'1 , Lambda':"1clmda",'u + Lambda': "uclmda"}
POPULATION_SIZES = [5, 10, 20, 100]


In [36]:
print("---------Evolutionary Strategies------------") 
print('---------- (1 + 1)-ES ----------')
for N in POPULATION_SIZES:
    nodes = problem(N,42)
    evolutionary_Strategy(nodes,N,mtype=STRATEGY_TYPE["1 + 1"])

print('---------- (1 + Lambda)-ES ----------')
for N in POPULATION_SIZES:
    nodes = problem(N,42)
    evolutionary_Strategy(nodes,N,mtype=STRATEGY_TYPE["1 + Lambda"])

print('---------- (1 , Lambda)-ES ----------')       
for N in POPULATION_SIZES:
    nodes = problem(N,42)
    evolutionary_Strategy(nodes,N,mtype=STRATEGY_TYPE["1 , Lambda"])

print('---------- (u + Lambda)-ES ----------')
for N in POPULATION_SIZES:
    nodes = problem(N,42)
    evolutionary_Strategy(nodes,N,mtype=STRATEGY_TYPE["u + Lambda"])

    

---------Evolutionary Strategies------------
---------- (1 + 1)-ES ----------
---------- (u + Lambda)-ES ----------
For N = 5 => the list has a weight W = 25 and we used 25 nodes to cover it
For N = 10 => the list has a weight W = 101 and we used 50 nodes to cover it
For N = 20 => the list has a weight W = 104 and we used 25 nodes to cover it
For N = 100 => the list has a weight W = 235 and we used 9 nodes to cover it
